In [4]:
# Data processing
import pandas as pd
from surprise import Reader
from surprise import Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [5]:
import os
os.getcwd()

'c:\\Users\\Manish Seal\\OneDrive - National University of Singapore\\Documents\\NUS stuff\\y3s2\\DSA4212\\collaborative_filtering\\src'

In [6]:
matrix = pd.read_pickle("../data/item_matrix.pkl")
matrix.head()

userID,0,4,5,7,14,20,31,33,40,46,...,199956,199969,199973,199974,199975,199976,199980,199988,199990,199996
songID,,,,,,,,,,,,,,,,,,,,,
2263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
3785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print(matrix.min().min(), matrix.max().max())

1.0 5.0


In [8]:
def get_data(matrix):
    matrix_filled = matrix.fillna(0)
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(matrix_filled.stack().reset_index(), reader)
    return data

In [9]:
# 5 fold cross-validation
algo = SVD()
cross_validate(algo, get_data(matrix), cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0851  1.0819  1.0841  1.0843  1.0835  1.0838  0.0011  
MAE (testset)     1.0352  1.0337  1.0347  1.0349  1.0345  1.0346  0.0005  
Fit time          37.61   45.15   44.42   45.02   45.09   43.46   2.94    
Test time         8.35    6.57    7.90    7.44    6.42    7.34    0.75    


{'test_rmse': array([1.0851248 , 1.08185686, 1.08408593, 1.08432413, 1.0835235 ]),
 'test_mae': array([1.03516296, 1.03368708, 1.03468644, 1.03488836, 1.03446644]),
 'fit_time': (37.612250089645386,
  45.152132511138916,
  44.42175245285034,
  45.01781916618347,
  45.094085693359375),
 'test_time': (8.352168083190918,
  6.57180380821228,
  7.899521112442017,
  7.439632177352905,
  6.415766954421997)}

In [10]:
data = get_data(matrix)
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.2)
# We'll use the famous SVD algorithm.
algo = SVD()
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.0841


1.084112642691491